In [ ]:
# default_exp models_resnet_3d

# Models using 3D convolutions

> This module focuses on preparing the data of the UCF101 dataset to be used with the core functions.

Refs.
[understanding-1d-and-3d-convolution](https://towardsdatascience.com/understanding-1d-and-3d-convolution-neural-network-keras-9d8f76e29610)

In [1]:
#export
import torch
import torch.nn as nn
import torchvision  # used to download the model
import torch.nn.functional as F
from torch.autograd import Variable
import math

In [2]:
#export
def conv3x3x3(in_channels, out_channels, stride=1):
    # 3x3x3 convolution with padding
    return nn.Conv3d(
        in_channels,
        out_channels,
        kernel_size=3,
        stride=stride,
        padding=1,
        bias=False)


def downsample_basic_block(x, planes, stride):
    out = F.avg_pool3d(x, kernel_size=1, stride=stride)
    zero_pads = torch.Tensor(
        out.size(0), planes - out.size(1), out.size(2), out.size(3),
        out.size(4)).zero_()
    
    if isinstance(out.data, torch.cuda.FloatTensor):
        zero_pads = zero_pads.cuda()

    out = Variable(torch.cat([out.data, zero_pads], dim=1))

    return out

In [3]:
#export
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3x3(in_channels, channels, stride)
        self.bn1 = nn.BatchNorm3d(channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(channels, channels)
        self.bn2 = nn.BatchNorm3d(channels)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

In [4]:
#export 
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv3d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm3d(planes)
        self.conv2 = nn.Conv3d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm3d(planes)
        self.conv3 = nn.Conv3d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm3d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

In [5]:
#export
class ResNet(nn.Module):

    def __init__(self, block, layers, sample_size,
                 sample_duration, shortcut_type='B',
                 num_classes=400):
        self.inplanes = 64
        super(ResNet, self).__init__()
                
        self.conv1 = nn.Conv3d(3,
                               64,
                               kernel_size=7,
                               stride=(1, 2, 2),
                               padding=(3, 3, 3),
                               bias=False)
        self.bn1  = nn.BatchNorm3d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], shortcut_type)
        self.layer2 = self._make_layer(block, 128, layers[1], shortcut_type, stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], shortcut_type, stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], shortcut_type, stride=2)
        last_duration = int(math.ceil(sample_duration / 16))
        last_size = int(math.ceil(sample_size / 32))
        self.avgpool = nn.AvgPool3d((last_duration, last_size, last_size), stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                m.weight = nn.init.kaiming_normal_(m.weight, mode='fan_out')
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, shortcut_type, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            if shortcut_type == 'A':
                downsample = partial(
                    downsample_basic_block,
                    planes=planes * block.expansion,
                    stride=stride)
            else:
                downsample = nn.Sequential(
                    nn.Conv3d(self.inplanes,
                                planes * block.expansion,
                                kernel_size=1,
                                stride=stride,
                                bias=False), 
                    nn.BatchNorm3d(planes * block.expansion))

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):        
        with torch.no_grad():
            h = self.conv1(x)
            h = self.bn1(h)
            h = self.relu(h)
            h = self.maxpool(h)

            h = self.layer1(h)
            h = self.layer2(h)
            h = self.layer3(h)
        h = self.layer4[0](h)
#         h = self.layer4(h)

        h = self.avgpool(h)

        h = h.view(h.size(0), -1)
        h = self.fc(h)

        return h

In [6]:
#export
class ResNet50_3D(nn.Module):
    def __init__(self, num_classes, **kwargs):
        super(ResNet50_3D, self).__init__()
     
        if 'model_pretrained' in kwargs.keys():
            print(f"ResNet50_3D is loading pretrained ResNet50 from {kwargs['model_pretrained']}")
            pretrained_resnet50 = torch.load('./model-pretrained/resnet-50-kinetics.pth', map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
            kwargs.pop('model_pretrained', None)
            resnet = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)   
                  
            keys = [k for k,v in pretrained_resnet50['state_dict'].items()]
            pretrained_state_dict = {k[7:]: v.cpu() for k, v in pretrained_resnet50['state_dict'].items()}
            resnet.load_state_dict(pretrained_state_dict)            
    
        else:
            resnet = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
        
        # chenage the last layer to match number of classes
        resnet.fc = nn.Linear(resnet.fc.weight.shape[1], num_classes)
                  
#        self.feature_extractor = nn.Sequential(*list(resnet.children())[:-1])
        self.feature_extractor = resnet
#         self.final = nn.Sequential(
#             nn.Linear(resnet.fc.in_features, num_classes),
#         )
        

    def forward(self, x):
        # The input x will now be size [batch_size, c, seq_len, h, w].
        #batch_size, c, h, w = x.shape
        #x = x.view(batch_size, c, h, w)
        x = self.feature_extractor(x)
        #x = x.view(batch_size, -1)
#         x = self.final(x)
        #x = x.view(batch_size, -1)
        return x
                

In [7]:
#export
def resnet10(**kwargs):
    """Constructs a ResNet-18 model.
    """
    model = ResNet(BasicBlock, [1, 1, 1, 1], **kwargs)
    return model


def resnet18(**kwargs):
    """Constructs a ResNet-18 model.
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    return model


def resnet34(**kwargs):
    """Constructs a ResNet-34 model.
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
    return model


def resnet50(**kwargs):
    """Constructs a ResNet-50 model.
    """
    print('function resnet50')
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    return model

def resnet101(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    return model


def resnet152(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    return model


def resnet200(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 24, 36, 3], **kwargs)
    return model

In [8]:
model = ResNet50_3D(num_classes=101, sample_size=224, sample_duration=16, model_pretrained='./model-pretrained/resnet-50-kinetics.pth')

ResNet50_3D is loading pretrained ResNet50 from ./model-pretrained/resnet-50-kinetics.pth


In [71]:
model

ResNet50_3D(
  (feature_extractor): ResNet(
    (conv1): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplac

In [72]:
model = resnet10(sample_size=224, sample_duration=16)

In [11]:
model

ResNet(
  (conv1): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(128, eps=1e-05, momentum=0.1, 

In [1]:
#hide
from nbdev.export import *
notebook2script()

Converted 01_dataset_ucf101.ipynb.
Converted 02_avi.ipynb.
Converted 04_data_augmentation.ipynb.
Converted 05_models.ipynb.
Converted 06_models-resnet_3d.ipynb.
Converted 07_utils.ipynb.
Converted 10_run-baseline.ipynb.
Converted 11_run-sequence-convlstm.ipynb.
Converted 12_run-sequence-3d.ipynb.
Converted Smaller UCF-dataset-Copy1.ipynb.
Converted Smaller UCF-dataset.ipynb.
Converted index.ipynb.
